In [1]:
import streamlit as st
import numpy as np
import cv2
import requests
from skimage import color
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import pytesseract
from langdetect import detect
from difflib import SequenceMatcher

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def load_pretrained_cnn_model():
    # Load pre-trained VGG16 model
    model = VGG16(weights='imagenet', input_shape=(224, 224, 3))  # Add input_shape argument
    return model

def predict_image_class(model, image_path):
    # Load and preprocess the image for VGG16
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Get predictions for the image
    predictions = model.predict(img_array)

    # Decode and print the top-3 predicted classes
    decoded_predictions = decode_predictions(predictions, top=3)[0]
    
    if not decoded_predictions:
        st.write("No predictions available.")
        return None, None
    
    label, confidence = decoded_predictions[0][1], decoded_predictions[0][2]
    st.write(f"{label}: {confidence:.2f}")
    return label, confidence

def retrieve_image(image_url):
    response = requests.get(image_url)
    image = np.asarray(bytearray(response.content), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

def perform_error_level_analysis(image):
    # Calculate the error level of the image
    error_map = np.abs(np.diff(image.astype(int), axis=2)).sum(axis=2)
    error_level = error_map.mean()
    return error_level

def perform_sift_feature_extraction(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)
    return keypoints, descriptors

def display_image(image, title="Image"):
    plt.figure(figsize=(7, 7))  # Adjust the figure size as needed
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    plt.show()

def display_image_with_regions(image, regions, title="Tampered Image with Regions"):
    image_copy = image.copy()
    for i, region in enumerate(regions):
        x, y, w, h = region
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image_copy, f"Region {i + 1}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    plt.figure(figsize=(7, 7))  # Adjust the figure size as needed
    plt.imshow(cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    st.pyplot()

def detect_tampered_regions(image):
    image_array = np.array(image)
    if len(image_array.shape) == 2:
        gray_image = image_array
    else:
        gray_image = color.rgb2gray(image_array)
    gray_image = (gray_image * 255).astype(np.uint8)
    _, binary = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    tampered_regions = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 100:  # Adjust the threshold as per your needs
            x, y, w, h = cv2.boundingRect(contour)
            tampered_regions.append((x, y, w, h))
    return tampered_regions

def analyze_metadata(image):
    info = {}
    height, width = image.shape[:2]
    info['ImageWidth'] = width
    info['ImageHeight'] = height
    return info

def detect_scribbling(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total_area = 0
    for contour in contours:
        total_area += cv2.contourArea(contour)
        image_area = image.shape[0] * image.shape[1]
        area_percentage = total_area / image_area
        scribbling_threshold = 0.1  # Adjust this threshold value as needed
        if area_percentage > scribbling_threshold:
            return True
        else:
            return False

def detect_digital_forgery(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 100, 200)
    dilated_edges = cv2.dilate(edges, None)
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx_polygon = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        if len(approx_polygon) < 5:
            return True

        return False

def detect_data_manipulation(image):
    mean_intensity = np.mean(image)
    std_intensity = np.std(image)
    intensity_threshold = 50  # Adjust this threshold value as needed
    if std_intensity > intensity_threshold:
        return True

    return False

def detect_whitener(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    histogram = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
    whitened_pixels_percentage = histogram[-1] / np.sum(histogram)
    whitening_threshold = 0.1  # Adjust this threshold value as needed
    if whitened_pixels_percentage > whitening_threshold:
        return True

    return False

def generate_noise_map(image):
    image_array = np.array(image)
    if len(image_array.shape) == 2:
        noise_map = np.copy(image_array)
    else:
        gray_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
        blurred_gray_image = cv2.GaussianBlur(gray_image, (11, 11), 0)
        noise_map = cv2.absdiff(gray_image, blurred_gray_image)
        return noise_map

def perform_ocr(image):
    text = pytesseract.image_to_string(image)
    return text

def language_and_font_analysis(image):
    text_representation = pytesseract.image_to_string(image)
    language = detect(text_representation)
    return language

def false_positive_reduction(tampered_regions, noise_map):
    filtered_regions = []
    for region in tampered_regions:
        x, y, w, h = region
        region_noise = noise_map[y:y+h, x:x+w]
        region_noise_pixels = np.count_nonzero(region_noise)
        if region_noise_pixels / (region_noise.shape[0] * region_noise.shape[1]) > 0.1:
            filtered_regions.append(region)

    return filtered_regions

def visualize_noise_map(image, noise_map):
    noise_map_rgb = cv2.cvtColor(noise_map, cv2.COLOR_GRAY2RGB)
    alpha = 0.6
    output = cv2.addWeighted(np.array(image), 1-alpha, noise_map_rgb, alpha, 0)
    display_image(output)

def compare_text(genuine_text, tampered_text):
    seq_matcher = SequenceMatcher(None, genuine_text, tampered_text)
    differences = list(seq_matcher.get_opcodes())
    for tag, i1, i2, j1, j2 in differences:
        if tag in ['insert', 'replace']:
            st.write(f'\033[1mTampered Text Differs at Position [{j1}:{j2}]: "{tampered_text[j1:j2]}"\033[0m')

def process_tampered_image(tampered_image_url):
    # Load pre-trained CNN model
    cnn_model = load_pretrained_cnn_model()

    # Retrieve the tampered image
    tampered_image = retrieve_image(tampered_image_url)

    # Save the tampered image for CNN processing
    cnn_image_path = "tampered_image_for_cnn.jpg"
    cv2.imwrite(cnn_image_path, tampered_image)
    predicted_label, confidence = predict_image_class(cnn_model, cnn_image_path)
    st.write(f"\033[1mPredicted class: {predicted_label}, Confidence: {confidence:.2f}\033[0m")

    # Perform error level analysis on the tampered image
    tampered_error_level = perform_error_level_analysis(np.array(tampered_image))

    tampered_keypoints, tampered_descriptors = perform_sift_feature_extraction(np.array(tampered_image))
    tampered_regions = detect_tampered_regions(tampered_image)

    metadata_tampered = analyze_metadata(tampered_image)

    noise_map = generate_noise_map(tampered_image)

    text_tampered = perform_ocr(tampered_image)

    language_tampered = language_and_font_analysis(tampered_image)

    tampered_regions_filtered = false_positive_reduction(tampered_regions, noise_map)

    # Display the tampered image with regions
    display_image_with_regions(tampered_image, tampered_regions_filtered)

    st.write("\033[1mError level of tampered image:\033[0m", tampered_error_level)
    tampered_image_types = []
    scribbling_regions = detect_scribbling(tampered_image)
    digital_forgery_regions = detect_digital_forgery(tampered_image)
    data_manipulation_regions = detect_data_manipulation(tampered_image)
    whitener_regions = detect_whitener(tampered_image)

    if scribbling_regions:
        tampered_image_types.append("Scribbling/Overwriting Forgery")
        st.write("\033[1mScribbling/Overwriting Forgery detected at regions:\033[0m", scribbling_regions)
    if digital_forgery_regions:
        tampered_image_types.append("Digital Forgery")
        st.write("\033[1mDigital Forgery detected at regions:\033[0m", digital_forgery_regions)
    if data_manipulation_regions:
        tampered_image_types.append("Data Manipulation Forgery")
        st.write("\033[1mData Manipulation Forgery detected at regions:\033[0m", data_manipulation_regions)
    if whitener_regions:
        tampered_image_types.append("Whitener Forgery")
        st.write("\033[1mWhitener Forgery detected at regions:\033[0m", whitener_regions)

    # Classification based on forgery detection
    if tampered_image_types:
        st.write("\033[1mClassification: Forged\033[0m")
    else:
        st.write("\033[1mClassification: Genuine\033[0m")

    # Predict the image class using the pre-trained CNN
    predict_image_class(cnn_model, cnn_image_path)

    # Display the tampered image with regions
    for region in tampered_regions_filtered:
        if isinstance(region, bool):  # Skip boolean values
            continue
        x, y, w, h = region
        cv2.rectangle(tampered_image, (x, y), (x + w, y + h), (0, 0, 255), 2)

    st.image(tampered_image, caption="Tampered Image", use_column_width=True)
    st.write("\033[1mError level of tampered image:\033[0m", tampered_error_level)
    st.write("\033[1mNumber of keypoints in tampered image:\033[0m", len(tampered_keypoints))
    st.write("\033[1mMetadata of tampered image:\033[0m", metadata_tampered)
    st.write("\033[1mText detected in tampered image:\033[0m", text_tampered)
    st.write("\033[1mLanguage of tampered image:\033[0m", language_tampered)

    visualize_noise_map(tampered_image, noise_map)

# Create a Streamlit app
st.title("Image Analysis App")

# Add a text input field for the user to enter the image URL
image_url = st.text_input("Enter the image URL:")

# Check if the user has entered a valid URL
if image_url:
    # Process the tampered image using the provided URL
    process_tampered_image(image_url)
else:
    st.write("Please enter a valid image URL.")


2023-12-08 15:24:08.048 
  command:

    streamlit run C:\Users\zzzri\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
